# Importamos librerias 

In [1]:
import findspark
findspark.init()


In [2]:
#from pyspark.sql import SparkSession #Para crear una sesión de Spark
from pyspark.sql.types import *  
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from collections import Counter #Para contar la frecuencia de los elementos de una lista
import json # Datos en formato JSON, pues los datos de BICIMAD vienen en ese formato
#from pprint import pprint # Para imprimir de manera legible los resultados
import datetime # Para poder trabajar con fechas 
import sys
import statistics

# Creamos sesión en spark

In [3]:
#spark = SparkSession.builder.getOrCreate() #Creamos la sesión
conf = SparkConf()\
        .setAppName('PracticaSpark')
sc = SparkContext(conf = conf)

# Abrimos el archivo correspondiente a junio de 2022


In [4]:
import json
with open(r"C:/Users/j4gon/Downloads/junio2019.json", encoding='latin1') as f:
    data = f.readlines()
     

rdd = sc.textFile(r"C:/Users/j4gon/Downloads/junio2019.json")

In [5]:
def FiletoDic(line):#Seleccionamos los datos que vamos a utilizar

  data=json.loads(line)
  id_bici = data['_id']
  usuario = data['user_day_code']
  salida = data['idunplug_station']
  llegada = data['idplug_station']
  tiempo = data['travel_time']
  edad = data['ageRange']
  
  return usuario,id_bici,salida,llegada,tiempo,edad

In [6]:
rdd1 = rdd.map(FiletoDic)
rdd.map(FiletoDic).take(1)

[('e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b',
  {'$oid': '5cf83b752f3843a016be4e2f'},
  90,
  66,
  219,
  0)]

In [7]:
rdd2 = sc.parallelize([('a',1),('b',4),('a',8),('b',5),('b',8)])
#rdd_estacion_origen = rdd2.groupByKey().mapValues(lambda x: sum(list(x)))
rdd_estacion_origen = rdd2.reduceByKey(lambda x, y: x + y)

                        
                          
                          
        

         

#rdd_estacion_destino = rdd1.map(lambda x: (x[3], 1))\
#                           .groupByKey()\
#                           .mapValues(sum)\
#                           .sortBy(lambda x: x[1], ascending=False)

rdd_estacion_origen.collect()
#rdd_estacion_destino.take(10)


[('a', 9), ('b', 17)]

In [8]:
rdd1 = rdd.map(FiletoDic)
rdd.map(FiletoDic).take(10)

[('e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b',
  {'$oid': '5cf83b752f3843a016be4e2f'},
  90,
  66,
  219,
  0),
 ('8a0c4123e924a50a958f51985eb71aea750fb072438035f149283490cc6bfaf4',
  {'$oid': '5cf83b762f3843a016be4e48'},
  71,
  136,
  359,
  4),
 ('a6a9c1f74a68496000542210abc4fc2eba79e2756ad5355a626632f7783dd418',
  {'$oid': '5cf83b762f3843a016be4e4f'},
  39,
  38,
  375,
  4),
 ('5706c0bd494acc02279d532821c9666b0e506d4f81c838ef3147b2e0abbe7156',
  {'$oid': '5cf83b762f3843a016be4e53'},
  66,
  90,
  264,
  5),
 ('eb1b6d32bd4add5d5ff91af72a38786d61075c090383a53882f33b0801eb0079',
  {'$oid': '5cf83b762f3843a016be4e54'},
  152,
  166,
  367,
  4),
 ('c2905f6038aa9523da6bc3222c63c2ec3269c69281bf34eac161eed1ae2ba5f4',
  {'$oid': '5cf83b762f3843a016be4e56'},
  55,
  53,
  174,
  5),
 ('f94420744ea060ac42ef3901cb251bb5b4ff63033be7bcb96d8764b968912691',
  {'$oid': '5cf83b762f3843a016be4e59'},
  133,
  129,
  308,
  0),
 ('e75471af2ea032a327f636e28b8603c4cddab9155948fca

# Obtenemos las estaciones de salida y llegada para cada usuario

In [9]:
def salidas(line):
    data    = FiletoDic(line)
    return data[0],data[2]

In [10]:
lista_salidas = rdd.map(salidas)
lista_salidas.take(10)

[('e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b', 90),
 ('8a0c4123e924a50a958f51985eb71aea750fb072438035f149283490cc6bfaf4', 71),
 ('a6a9c1f74a68496000542210abc4fc2eba79e2756ad5355a626632f7783dd418', 39),
 ('5706c0bd494acc02279d532821c9666b0e506d4f81c838ef3147b2e0abbe7156', 66),
 ('eb1b6d32bd4add5d5ff91af72a38786d61075c090383a53882f33b0801eb0079', 152),
 ('c2905f6038aa9523da6bc3222c63c2ec3269c69281bf34eac161eed1ae2ba5f4', 55),
 ('f94420744ea060ac42ef3901cb251bb5b4ff63033be7bcb96d8764b968912691', 133),
 ('e75471af2ea032a327f636e28b8603c4cddab9155948fca85e82ec2f73c2bd1a', 153),
 ('c0d73ee753773a3bfcc1483366a6ee68be9524dafbf65e81d31f4509d73304ac', 44),
 ('47ecd557f21ca7aee6235dc015c6eaa7cea5e99e3aa8fbdfac8a308bb093f552', 85)]

In [11]:
def llegadas(line):
    data    = FiletoDic(line)
    return data[0],data[3]

In [12]:
lista_llegadas = rdd.map(llegadas)
lista_llegadas.take(10)

[('e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b', 66),
 ('8a0c4123e924a50a958f51985eb71aea750fb072438035f149283490cc6bfaf4', 136),
 ('a6a9c1f74a68496000542210abc4fc2eba79e2756ad5355a626632f7783dd418', 38),
 ('5706c0bd494acc02279d532821c9666b0e506d4f81c838ef3147b2e0abbe7156', 90),
 ('eb1b6d32bd4add5d5ff91af72a38786d61075c090383a53882f33b0801eb0079', 166),
 ('c2905f6038aa9523da6bc3222c63c2ec3269c69281bf34eac161eed1ae2ba5f4', 53),
 ('f94420744ea060ac42ef3901cb251bb5b4ff63033be7bcb96d8764b968912691', 129),
 ('e75471af2ea032a327f636e28b8603c4cddab9155948fca85e82ec2f73c2bd1a', 169),
 ('c0d73ee753773a3bfcc1483366a6ee68be9524dafbf65e81d31f4509d73304ac', 129),
 ('47ecd557f21ca7aee6235dc015c6eaa7cea5e99e3aa8fbdfac8a308bb093f552', 133)]

# Ahora comprobemos aquellos usuario realizan un ciclo

Un usuario realiza un ciclo si la estación de salida y de llegada coinciden, idependientemente de si en las estaciones que pare entre medias coja otra bici.

In [13]:
def ciclo(line):
    data = FiletoDic(line)
    if data[2] == data[3]: 
        return (True, tiempo)
    else:
        return False

In [18]:
rdd1.filter(lambda x : x[2] == x[3]).take(2)

[('bdfbb7774e97c37e95c79efa9e5f1cba2f6672dbf1f71c1faa2bc89e9e423ae0',
  {'$oid': '5cf83b762f3843a016be4ea0'},
  83,
  83,
  357,
  0),
 ('c199289b53d6909ee2943de60baf3fd465f9ce30251683764ed30febe54b29ef',
  {'$oid': '5cf83b762f3843a016be4eb4'},
  48,
  48,
  520,
  0)]

In [15]:
usuarios_ciclo = rdd1.filter(lambda x: x[2] == x[3])

Obtenemos la el tiempo medio de los usuarios que realizan un ciclo

In [16]:
media_ciclo = usuarios_ciclo.map(lambda x : x[4]).mean()

In [17]:
media_ciclo

1636.1146547502462

Ahora veamos el número de que realiza cada usuario que hace un ciclo

In [20]:
prueba = sc.parallelize([('a','cola'),('b','hola'),('a','colita'),('a','cola')])
prueba.groupByKey()\
      .mapValues(lambda x: len(x)).collect()

[('a', 3), ('b', 1)]

In [23]:
num_bicis_ciclo = usuarios_ciclo.map(lambda x: (x[0],x[1]))\
                                .groupByKey()\
                                .mapValues(lambda x: len(x))
num_bicis_ciclo.take(5)

[('c199289b53d6909ee2943de60baf3fd465f9ce30251683764ed30febe54b29ef', 1),
 ('72c4d5ae9ffe74250e24f097428e68286a2508dfa3c5a3edb18a5778c917680b', 1),
 ('4cdb1b6ad690fd22b6ab696e30ec0df064c664907ca3b8cf9f0d03d0dbca0df0', 1),
 ('f7bd6a6fc4d1e9877a408549c6e6c5472950149a038efd55fec7e246c1d2415f', 1),
 ('a25b20b6be2e1edd62999be61293d1694a281f6a981063792d5e915f2a1165a1', 1)]

Ahora veamos de los usuarios que realizan un ciclo, cuales realizan un camino, que basicamente es 
comprobar que usuarios usan la misma bici en todo el trayecto

In [24]:
def es_camino(lista_bicis):
    a = lista_bicis[0]
    camino = True
    if len(lista_bicis) == 1:
        return True
    else:
        for b in range(1,len(lista_bicis)):
            if a != lista_bicis[b]:
                return False
    return camino
            

In [25]:
usuarios_camino = usuarios_ciclo.map(lambda x: (x[0],x[1]))\
                                .groupByKey()\
                                .mapValues(rdd.reduce(compare_elements))\
                                .take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 111) (LAPTOP-1KV5MILP.home executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark-3.4.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 830, in main
  File "C:\spark-3.4.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 822, in process
  File "C:\spark-3.4.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\spark-3.4.0-bin-hadoop3\python\pyspark\rdd.py", line 2830, in takeUpToNumLeft
    yield next(iterator)
          ^^^^^^^^^^^^^^
  File "C:\spark-3.4.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\util.py", line

In [ ]:
usuarios_camino